Temp Formulation

<h4> Sets and Indicies </h4>

$\mathcal{L}$: Hidden layers of the Neural Network. $|\mathcal{L}| = \textbf{L}$ <br>
$l$: An arbitrary element in $\mathcal{L}$ <br>

$\mathcal{N}$: Neurons in the hidden layers of the Neural Network. $|\mathcal{N}| = \textbf{N}$ <br>
$n$: An arbitrary element in $\mathcal{N}$ <br>

$\mathcal{P}$: Indicies of the Datapoints used to evaluate the model $|\mathcal{P}| = \textbf{P}$ <br>
$p$: An arbitrary element in $\mathcal{P}$ <br>


<h4> Data </h4>

$W$: Weights of the neurons in the hidden layers of the neural network. <br>
$w_{ln}; \; l \in \mathcal{L},\; n \in \mathcal{N}$: An index in $W$. <br>

$D$: Datapoints for evaluating the neural network. <br>
$d_{p}; \; p \in \mathcal{P}$: An index in $D$. <br>

$Y$: True labels of the datapoints for evaluating the neural network. <br>
$y_{p}; \; p \in \mathcal{P}$: An index in $Y$. <br>

<h4> Decision Variable </h4>

$x_{ln}; \; l \in \mathcal{L},\; n \in \mathcal{N}$: Whether this weight is chosen or not.  

<h4> Function </h4>

\begin{align*}

& \mathrm{minimize}{\sum_{i=1}^{\textbf{L}}\sum_{j=1}^{\textbf{N}}{x_{ij}} - \sum_{u=1}^{\textbf{P}}{ y_{u} \log{ f_{\hat{W}}(d_u) } } }  \\ \\
\text{Where} \\
& f_{\theta} \; \text{is the output of the model with weights } \theta \\
& \hat{W} \; = x_{ij} w_{ij} \;;\; \forall i \in \mathcal{N} ,\; \forall j \in \mathcal{L} \\

\text{Subject to} \\
& \text{No Constraints for now.}

\end{align*}

In [1]:
import numpy as np
import glob2 as glob
import pandas as pd
import copy
from tqdm import tqdm

In [2]:
files = glob.glob('1K_halfK_model_weights_3_layers/*.npy')

weights_dict = {'bias' : {}, 'weight' : {}}

for file in files:
    layer_name = file.split('/')[-1].split('.')[0]
    if layer_name.split('_')[-1] == 'bias':
        weights_dict['bias'].update({layer_name.split('_')[0]: np.load(file)})
    else:
        weights_dict['weight'].update({layer_name.split('_')[0]: np.load(file)})

test_df = pd.read_csv('mnist_test.csv')
order= ['fc1', 'fc2', 'fc3']

In [8]:
def choose_neurons_numeric(chosen_neurons, _weights_dict):
    
    weights_dict_ = copy.deepcopy(_weights_dict)
        
    weights_dict_['weight']['fc1'] = weights_dict_['weight']['fc1'][chosen_neurons[0], :]
    weights_dict_['bias']['fc1'] = weights_dict_['bias']['fc1'][chosen_neurons[0]]

    weights_dict_['weight']['fc2'] = weights_dict_['weight']['fc2'][chosen_neurons[1], :][:, chosen_neurons[0]]
    weights_dict_['bias']['fc2'] = weights_dict_['bias']['fc2'][chosen_neurons[1]]

    weights_dict_['weight']['fc3'] = weights_dict_['weight']['fc3'][: ,chosen_neurons[1]]
    
    return weights_dict_


def choose_neurons_expression(chosen_neurons, _weights_dict):
    '''
    Please execuse this god awful code. I am sorry.
    docplex expression is so much of a pain in the ass to work with, that
    running it on "for" loops is faster than np.dot or than "@"   
    '''
    weights_dict_ = copy.deepcopy(_weights_dict)
        
    #############
    ## LAYER 1 ##
    #############

    # Adding expression to the weights of Layer 1
    print('Adding expression to the weights of Layer 1')
    new_fc = []
    for i in tqdm(range(weights_dict_['weight']['fc1'].shape[0])):
        new_fc_row = []
        for j in range(weights_dict_['weight']['fc1'].shape[1]):
            new_fc_row.append(chosen_neurons[0][i] * weights_dict_['weight']['fc1'][i][j])
        new_fc.append(new_fc_row)

    weights_dict_['weight']['fc1'] = np.array(new_fc)

    # Adding expression to the Bias of Layer 1
    print('Adding expression to the bias of Layer 1')
    new_fc = []
    for i in tqdm(range(weights_dict_['bias']['fc1'].shape[0])):
        new_fc.append(chosen_neurons[0][i] * weights_dict_['bias']['fc1'][i])
    
    weights_dict_['bias']['fc1'] = np.array(new_fc)


    #############
    ## LAYER 2 ##
    #############

    # Adding expression to the weights of Layer 2
    print('Adding expression to the weights of Layer 2')
    
    new_fc = weights_dict_['weight']['fc2'].copy().astype(object)
    for i in tqdm(range(weights_dict_['weight']['fc2'].shape[0])):
        for j in range(weights_dict_['weight']['fc2'].shape[1]):
            new_fc[i][j] *= (chosen_neurons[1][i] * chosen_neurons[0][j])

    weights_dict_['weight']['fc2'] = new_fc

    # Adding expression to the Bias of Layer 2
    print('Adding expression to the bias of Layer 2')
    new_fc = []
    for i in tqdm(range(weights_dict_['bias']['fc2'].shape[0])):
        new_fc.append(chosen_neurons[1][i] * weights_dict_['bias']['fc2'][i])
    
    weights_dict_['bias']['fc2'] = np.array(new_fc)

    #############
    ## LAYER 3 ##
    #############
    
    # Adding expression to the weights of Layer 3
    print('Adding expression to the weights of Layer 3')
    
    new_fc = weights_dict_['weight']['fc3'].copy().astype(object)
    for i in tqdm(range(weights_dict_['weight']['fc3'].shape[0])):
        for j in range(weights_dict_['weight']['fc3'].shape[1]):
            new_fc[i][j] *= chosen_neurons[1][j]

    weights_dict_['weight']['fc3'] = new_fc

    return weights_dict_

def predict_expressions(expression_matrix, weights_dict_):

    x = np.dot(weights_dict_['weight']['fc1'], expression_matrix.T) + weights_dict_['bias']['fc1']
    # x = np.maximum(0, x)

    print(x)
    
    for i in x:
        print(i)

    print(x.shape)

    print('Done 1 layer')

    x = np.dot(weights_dict_['weight']['fc2'], x.T) + weights_dict_['bias']['fc2']
    # x = np.maximum(0, x)


    print('Done 2 layer')

    x = np.dot(weights_dict_['weight']['fc3'], x.T) + weights_dict_['bias']['fc3']
    # x = np.maximum(0, x)

    print('Done 3 layer')

    x /= np.sum(x) # softmax

    return x

def predict_numeric(img_df_row, weights_dict_):

    x = np.dot(weights_dict_['weight']['fc1'], img_df_row.T) + weights_dict_['bias']['fc1']
    x = np.maximum(0, x)

    x = np.dot(weights_dict_['weight']['fc2'], x.T) + weights_dict_['bias']['fc2']
    x = np.maximum(0, x)

    x = np.dot(weights_dict_['weight']['fc3'], x.T) + weights_dict_['bias']['fc3']
    x = np.maximum(0, x)

    x /= np.sum(x) # softmax

    return x

def print_neurons_shape(weights_dict_):
    print('Layer Number\t|\t Weights Shape\t|\t Bias Shape')
    print('-'*60)
    print('Layer 1  \t|\t', weights_dict_['weight']['fc1'].shape, '\t|\t' , weights_dict_['bias']['fc1'].shape)
    print('Layer 2  \t|\t', weights_dict_['weight']['fc2'].shape, '\t|\t' , weights_dict_['bias']['fc2'].shape)
    print('Layer 3  \t|\t', weights_dict_['weight']['fc3'].shape, '\t|\t' , weights_dict_['bias']['fc3'].shape)
    
def cross_entropy_loss(label, y_hat):
    y = np.zeros(10)
    y[label] = 1
    return -np.sum(y * np.log(y_hat))

print_neurons_shape(weights_dict)

Layer Number	|	 Weights Shape	|	 Bias Shape
------------------------------------------------------------
Layer 1  	|	 (1024, 784) 	|	 (1024,)
Layer 2  	|	 (512, 1024) 	|	 (512,)
Layer 3  	|	 (10, 512) 	|	 (10,)


In [4]:
### VALIDATION DRAFT: DO NOT REMOVE AND DO NOT RUN!!! ###

# # chosen_fc1 = np.random.randint(0, 2, 2048)
# # chosen_fc2 = np.random.randint(0, 2, 2048)

# # print(chosen_fc1.sum(), chosen_fc2.sum())

# # expressioned_weights = choose_neurons_expression([chosen_fc1, chosen_fc2], weights_dict)

# # 1012 1032

# print((expressioned_weights['weight']['fc1'].sum(axis = 0) != 0).sum() , (expressioned_weights['weight']['fc1'].sum(axis = 1) != 0).sum())
# print((expressioned_weights['weight']['fc2'].sum(axis = 0) != 0).sum() , (expressioned_weights['weight']['fc2'].sum(axis = 1) != 0).sum())
# print((expressioned_weights['weight']['fc3'].sum(axis = 0) != 0).sum() , (expressioned_weights['weight']['fc3'].sum(axis = 1) != 0).sum())

### DoCplex

In [4]:
from docplex.mp.model import Model

m = Model()

X_fc1 = m.binary_var_list(1024, name='Layer 1 Neurons')
X_fc2 = m.binary_var_list(512, name='Layer 2 Neurons')

In [5]:
# This can take up-to 5 buisness days to run.
# For some reason, docplex.Expression is as optimized as my ass after eating Tarroub, i.e., not optimized
expressioned_weights = choose_neurons_expression([X_fc1, X_fc2], weights_dict)

Adding expression to the weights of Layer 1


100%|██████████| 1024/1024 [00:07<00:00, 139.61it/s]


Adding expression to the bias of Layer 1


100%|██████████| 1024/1024 [00:00<00:00, 75442.95it/s]


Adding expression to the weights of Layer 2


100%|██████████| 512/512 [00:14<00:00, 35.16it/s]


Adding expression to the bias of Layer 2


100%|██████████| 512/512 [00:00<00:00, 136608.37it/s]


Adding expression to the weights of Layer 3


100%|██████████| 10/10 [00:00<00:00, 370.39it/s]


In [6]:
data = test_df.drop(columns=['label'])
true_class = test_df['label']

In [9]:
for idx in range(len(data)):
    y_hat = predict_expressions(data.iloc[idx], expressioned_weights)
    
    print(cross_entropy_loss(true_class[idx], y_hat))
    break

[-116.923Layer 1 Neurons_0 27.045Layer 1 Neurons_1
 -137.335Layer 1 Neurons_2 ... 72.193Layer 1 Neurons_1021
 -48.404Layer 1 Neurons_1022 62.266Layer 1 Neurons_1023]
-116.923Layer 1 Neurons_0
27.045Layer 1 Neurons_1
-137.335Layer 1 Neurons_2
-61.671Layer 1 Neurons_3
48.579Layer 1 Neurons_4
4.165Layer 1 Neurons_5
7.654Layer 1 Neurons_6
-11.956Layer 1 Neurons_7
9.690Layer 1 Neurons_8
9.656Layer 1 Neurons_9
-19.380Layer 1 Neurons_10
15.893Layer 1 Neurons_11
-103.105Layer 1 Neurons_12
81.863Layer 1 Neurons_13
-35.738Layer 1 Neurons_14
21.758Layer 1 Neurons_15
118.994Layer 1 Neurons_16
-58.876Layer 1 Neurons_17
95.387Layer 1 Neurons_18
66.357Layer 1 Neurons_19
2.252Layer 1 Neurons_20
12.836Layer 1 Neurons_21
-128.048Layer 1 Neurons_22
-194.675Layer 1 Neurons_23
19.416Layer 1 Neurons_24
-92.941Layer 1 Neurons_25
81.874Layer 1 Neurons_26
10.315Layer 1 Neurons_27
-170.249Layer 1 Neurons_28
59.431Layer 1 Neurons_29
51.130Layer 1 Neurons_30
63.524Layer 1 Neurons_31
-119.749Layer 1 Neurons_32
18.

SystemError: <class 'type'> returned a result with an exception set

In [155]:

number_of_nuerons_used = m.sum(X_fc1) + m.sum(X_fc2)



# loss = cross_entropy_loss(X_fc1)

# m.minimize(number_of_nuerons_used + )

In [158]:
expressioned_weights['weight']['fc1'][0]

docplex.mp.LinearExpr(-0.021Layer 1 Neurons_0-0.013Layer 1 Neurons_1+0.029Layer 1 Neurons_2-0.018Layer 1 Neurons_3+0.033Layer 1 Neurons_4+0.007Layer 1 Neurons_5-0.011Layer 1 Neurons_6-0.025Layer 1 Neurons_7-0.006Layer 1 Neurons_8+0.020Layer 1 Neurons_9+0.033Layer 1 Neurons_10-0.003Layer 1 Neurons_11-0.024Layer 1 Neurons_12+0.001Layer 1 Neurons_13+0.033Layer 1 Neurons_14+0.008Layer 1 Neurons_15-0.014Layer 1 Neurons_16-0.018Layer 1 Neurons_17+0.025Layer 1 Neurons_18+0.026Layer 1 Neurons_19+0.031Layer 1 Neurons_20+0.019Layer 1 Neurons_21+0.016Layer 1 Neurons_22-0.032Layer 1 Neurons_23+0.025Layer 1 Neurons_24-0.032Layer 1 Neurons_25+0.018Layer 1 Neurons_26-0.008Layer 1 Neurons_27-0.020Layer 1 Neurons_28-0.017Layer 1 Neurons_29+0.008Layer 1 Neurons_30-0.033Layer 1 Neurons_31-0.018Layer 1 Neurons_32+0.023Layer 1 Neurons_33-0.003Layer 1 Neurons_34+0.014Layer 1 Neurons_35-0.019Layer 1 Neurons_36+0.028Layer 1 Neurons_37+0.007Layer 1 Neurons_38-0.016Layer 1 Neurons_39-0.026Layer 1 Neurons_40-0.0